In [3]:
# Paso 2: Configurar e importar PySpark
from pyspark.sql import SparkSession
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
import matplotlib.pyplot as plt

In [4]:
# Crea una sesión de Spark
spark = SparkSession.builder \
    .appName("BicicletasDatos") \
    .getOrCreate()


In [5]:
from google.colab import files
uploaded = files.upload()

Saving 2025_02.csv to 2025_02 (1).csv


In [6]:
# Nota: Cambia "2025_02.csv" por el nombre correcto si difiere
df_spark = spark.read.csv("2025_02.csv", header=True, inferSchema=True)

In [7]:
# Opcional: Muestra el esquema inferido
df_spark.printSchema()

root
 |-- Genero_Usuario: string (nullable = true)
 |-- Edad_Usuario: string (nullable = true)
 |-- Bici: integer (nullable = true)
 |-- Ciclo_Estacion_Retiro: string (nullable = true)
 |-- Fecha_Retiro: string (nullable = true)
 |-- Hora_Retiro: timestamp (nullable = true)
 |-- Ciclo_EstacionArribo: string (nullable = true)
 |-- Fecha_Arribo: string (nullable = true)
 |-- Hora_Arribo: timestamp (nullable = true)



In [8]:
# Convertir el DataFrame de Spark a uno de pandas para el procesamiento
df = df_spark.toPandas()


In [9]:
# Convertir género a valores numéricos
le = LabelEncoder()
df['Genero_Usuario'] = le.fit_transform(df['Genero_Usuario'])  # M a 1, F a 0

In [12]:
import pandas as pd
# Asegúrate de que la columna 'Hora_Retiro' esté en formato datetime
df['Hora_Retiro'] = pd.to_datetime(df['Hora_Retiro'], format='%H:%M:%S', errors='coerce')

# Crear una variable objetivo (1 si usó bicicleta de noche, 0 de lo contrario)
# Comprobar si 'Hora_Retiro' es un Timestamp y comparar con el umbral de hora
df['Usa_Bici_Noche'] = df['Hora_Retiro'].apply(lambda x: 1 if x.time() >= pd.to_datetime('22:00:00').time() else 0)

In [17]:
# Ver los valores únicos en la columna 'Genero_Usuario' y 'Edad_Usuario'
print("Valores únicos de 'Genero_Usuario':", df['Genero_Usuario'].unique())
print("Valores únicos de 'Edad_Usuario':", df['Edad_Usuario'].unique())

# Reemplazar valores 'NULL' por NaN para facilitar la limpieza
df.replace('NULL', pd.NA, inplace=True)

# Convertir la columna 'Edad_Usuario' a numérica (si no está ya en formato numérico)
df['Edad_Usuario'] = pd.to_numeric(df['Edad_Usuario'], errors='coerce')

# Eliminar filas con NaN en características y etiqueta
df.dropna(subset=['Genero_Usuario', 'Edad_Usuario', 'Usa_Bici_Noche'], inplace=True)

# Ver cuántas filas quedan después de la limpieza
print(f'Número total de filas después de la limpieza: {df.shape[0]}')

Valores únicos de 'Genero_Usuario': [2 1 4 0 3]
Valores únicos de 'Edad_Usuario': ['43' '29' '22' '46' '23' '37' '21' '18' '28' '20' '27' '24' '62' '31'
 '25' '35' '30' '38' '45' '34' '36' '33' '39' '41' '53' '32' '19' '26'
 '51' '54' '58' '60' '40' '50' '69' '57' '42' '55' '44' '66' '48' '47'
 '61' '70' '49' '52' '59' '56' '64' '72' '73' '17' '63' '68' '100' '82'
 '65' '67' '91' '76' '71' '75' '77' '74' '102' '79' '80' '83' '16' '94'
 '78' 'NULL' '124' '99' '90' '101' '81' '85' '89' '95' '84' '143']
Número total de filas después de la limpieza: 1745522


In [18]:
# Asegúrate de que las columnas no tengan valores erróneos
X = df[['Genero_Usuario', 'Edad_Usuario']]
y = df['Usa_Bici_Noche']

In [19]:
# Dividir el conjunto de datos en conjunto de entrenamiento y prueba de nuevo, si es necesario
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [20]:
# Create and Train the model
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [21]:
# Test the mdel
accuracy = model.score(X_test, y_test)
print(f'Accuracy del modelo: {accuracy * 100:.2f}%')

Accuracy del modelo: 96.05%


In [22]:
# Realizar predicciones
predicciones = model.predict(X_test)
print(f'Predicciones: {predicciones}')

Predicciones: [0 0 0 ... 0 0 0]


In [27]:
import plotly.express as px

# Calcular el DataFrame de la matriz de confusión
cm = confusion_matrix(y_test, predicciones)
cm_df = pd.DataFrame(cm, index=['No Usa Bici de Noche', 'Usa Bici de Noche'],
                     columns=['No Usa Bici de Noche', 'Usa Bici de Noche'])

# Crear el gráfico
fig = px.imshow(cm_df, text_auto=True, color_continuous_scale='Greens',
                labels=dict(x="Predicción", y="Real", color="Conteo"),
                x=['No Usa Bici de Noche', 'Usa Bici de Noche'],
                y=['No Usa Bici de Noche', 'Usa Bici de Noche'])
fig.update_layout(title='Matriz de Confusión')
fig.show()

In [28]:
# Calcular y mostrar el classification report
report = classification_report(y_test, predicciones)
print("Classification Report:\n", report)


Classification Report:
               precision    recall  f1-score   support

           0       0.96      1.00      0.98    335301
           1       0.00      0.00      0.00     13804

    accuracy                           0.96    349105
   macro avg       0.48      0.50      0.49    349105
weighted avg       0.92      0.96      0.94    349105



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [32]:
# Crear un DataFrame para la comparativa
comparativa_df = pd.DataFrame({
    'Real': y_test.reset_index(drop=True),    # Datos reales
    'Predicted': predicciones                 # Predicciones del modelo
})


In [33]:
# Texto explicativo sobre la tabla
print("Comparison table:")
print("Note: In this table, the value '0' indicates that the user does NOT use the bike at night., "
      "while the value '1' indicates that the user DOES use the bicycle at night.\n")

# Mostrar la tabla comparativa
comparativa_df.head(20)

Comparison table:
Note: In this table, the value '0' indicates that the user does NOT use the bike at night., while the value '1' indicates that the user DOES use the bicycle at night.



,Real,Predicted
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
5,0,0
6,0,0
7,0,0
8,0,0
9,0,0


In [34]:
!pip install imbalanced-learn
from imblearn.over_sampling import RandomOverSampler

# 1. Definir el oversampler
ros = RandomOverSampler(random_state=42)

# 2. Ajustar y transformar las características y las etiquetas
X_resampled, y_resampled = ros.fit_resample(X, y)

# 3. Dividir el conjunto de datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# 4. Crear y entrenar el modelo
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# 5. Evaluar el modelo
accuracy = model.score(X_test, y_test)
print(f'Accuracy del modelo con oversampling: {accuracy * 100:.2f}%')

# 6. Realizar predicciones
predicciones = model.predict(X_test)

# 7. Calcular y mostrar el classification report
report = classification_report(y_test, predicciones)
print("Classification Report con oversampling:\n", report)

Accuracy del modelo con oversampling: 55.85%
Classification Report con oversampling:
               precision    recall  f1-score   support

           0       0.57      0.46      0.51    335685
           1       0.55      0.65      0.60    335009

    accuracy                           0.56    670694
   macro avg       0.56      0.56      0.55    670694
weighted avg       0.56      0.56      0.55    670694



In [36]:
from imblearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import cross_val_score
import numpy as np

# Definir el modelo
model = RandomForestClassifier(random_state=42)

# Definir RandomOverSampler
ros = RandomOverSampler(random_state=42)

# Crear una pipeline con el oversampling y el modelo usando imblearn
pipeline = Pipeline(steps=[('oversampler', ros), ('classifier', model)])

# Realizar la validación cruzada
scores = cross_val_score(pipeline, X, y, cv=5, scoring='f1_weighted')

# Mostrar los resultados
print(f'F1-Score de validación cruzada (weighted): {scores.mean():.2f} ± {scores.std():.2f}')

F1-Score de validación cruzada (weighted): 0.61 ± 0.02


In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'classifier__n_estimators': [100, 200],
    'classifier__max_depth': [None, 10, 20, 30],
    'classifier__min_samples_split': [2, 5, 10],
    'classifier__min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='f1_weighted')
grid_search.fit(X, y)

print("Mejores parámetros:", grid_search.best_params_)
print("Mejor F1-Score:", grid_search.best_score_)